## This notebook retrieves 2-meter temperature data from the High Resolution Rapid Refresh modeling system.
It is used to fetch 2mT for comparison to LST.

We use the Herbie library to access HRRR data.
https://herbie.readthedocs.io/en/latest/index.html

First install HErbie using conda:
>>conda install -c conda-forge herbie-data

Check the settings especially for file download location:

Some default settings are set in the config.toml file. This 
file is automatically created the first time you import a 
Herbie module and is located at 
    ${HOME}/.config/herbie/config.toml 
The configuration file is written in TOML format.

In [1]:
from herbie import Herbie

 ╭─────────────────────────────────────────────────╮
 │ I'm building Herbie's default config file.      │
 ╰╥────────────────────────────────────────────────╯
 👷🏻‍♂️
 ╭─────────────────────────────────────────────────╮
 │ You're ready to go.                             │
 │ You may edit the config file here:              │
 │ /Users/tylercruickshank/.config/herbie/config.toml   │
 ╰╥────────────────────────────────────────────────╯
 👷🏻‍♂️


In [ ]:
# Set up a Herbie object .
H = Herbie(
    "2021-08-01 00:00",  # model run date 2021-08-01T00:00:00Z
    model="hrrr",  # model name
    product="sfc",  # model product name (model dependent)
    fxx=0,  # forecast lead time
)

H.grib

# Get an inventory of all variables available.
H.inventory()

# We can search for certain variables.
# All fields at 2-m above ground level
H.inventory(searchString=":2 m above ground")

# Get 2mT.
H.inventory(searchString=":TMP:2 m")

# Download data and read with xarray.
ds = H.xarray(":TMP:2 m", remove_grib=False)
ds

# Plot
ds.t2m.plot()

Now we'll get multiple days of HRRR data.  
We want daily 00z for the month of August 2021.
We use FastHerbie to do ths.

In [ ]:
from herbie import FastHerbie
import pandas as pd

site_name = 'ames'
lat = 42
lon = -93.7
start_date = "2020-04-01 00:00"

# Create a range of dates
DATES = pd.date_range(
    start=start_date,
    periods=29,
    freq="1D",
)

# Define forecast lead time (or analysis).
fxx = range(0,1)

# Make a FastHerbie Object.
FH = FastHerbie(DATES, model="hrrr", fxx=fxx)
# The list of Herbie objects is stored in the objects property.
FH.objects

In [ ]:
# Read a subset of the data with xarray.
ds = FH.xarray("TMP:2 m", remove_grib=False)
ds

In [ ]:
# Get value nearest single point
# -120.21102905273436,36.535019209789
ds_point = ds.herbie.nearest_points(points=(lon, lat))
ds_point.t2m.plot()